# IF3270 Pembelajaran Mesin | Tugas Besar - Feedforward Neural Network

Group Members:
- Maximilian Sulistiyo (13522061)
- Marvel Pangondian (13522075)
- Abdullah Mubarak (13522101)

In this project, we implement a custom built Feedforward Neural Network with no high-level libraries. The goal in this project is to be able to create a custom FFNN that is able to specify the type of activation function on each layer, the type of loss function, and how many neurons in each layer. We will also compare our algorithm with other built in algorithm (the sklearn MLP)

## Import Libraries

In [1]:
import numpy as np
import pandas as pd
from ann import NeuralNetwork, one_hot, get_accuracy
from dense_layer import DenseLayer
from activations import *
from visualizer import visualize_ann
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from activations import tanh, sigmoid, relu


## Import Dataset

### Fetch dataset

In [2]:
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)

In [3]:
X.shape

(70000, 784)

In [4]:
y.shape

(70000,)

### Copy dataset and normalize

In [5]:
X_original = X.copy()
X_original = X_original/255.0
y_original = y.copy()
y_original = y_original.astype(int)

### Split dataset

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_original, y_original, train_size=60000, test_size=10000, stratify=y)

In [7]:
y_train_oh = one_hot(y_train)
y_test_oh = one_hot(y_test)

## Example of Training Model

In [8]:
model = NeuralNetwork('mse')
model.add_layer(DenseLayer(output_size=128, activation=relu, init="Xavier"))
model.add_layer(DenseLayer(output_size=64, activation=relu, init="Xavier"))
model.add_layer(DenseLayer(output_size=10, activation=sigmoid, init="Xavier"))

history = model.train(
    X_train,
    y_train,
    epochs=10,
    batch_size=64,
    learning_rate=0.05,
    isOne_hot=True,
    verbose=1,
    validation_data=(X_test, y_test)
)

[=================================================>] 100%
Epoch 1/10 - 2.90s - loss: 0.0949 - accuracy: 0.4680 - val_loss: 0.0786 - val_accuracy: 0.4671
[=================================================>] 100%
Epoch 2/10 - 2.61s - loss: 0.0670 - accuracy: 0.6755 - val_loss: 0.0564 - val_accuracy: 0.6778
[=================================================>] 100%
Epoch 3/10 - 2.47s - loss: 0.0491 - accuracy: 0.7591 - val_loss: 0.0437 - val_accuracy: 0.7596
[=================================================>] 100%
Epoch 4/10 - 2.67s - loss: 0.0398 - accuracy: 0.8405 - val_loss: 0.0366 - val_accuracy: 0.8408
[=================================================>] 100%
Epoch 5/10 - 3.09s - loss: 0.0335 - accuracy: 0.8656 - val_loss: 0.0311 - val_accuracy: 0.8665
[=================================================>] 100%
Epoch 6/10 - 2.94s - loss: 0.0288 - accuracy: 0.8760 - val_loss: 0.0272 - val_accuracy: 0.8751
[=================================================>] 100%
Epoch 7/10 - 2.52s - los

In [9]:
predictions = model.predict(X_test)
pred_classes = np.argmax(predictions, axis=1)
accuracy = accuracy_score(pred_classes, y_test)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.8965


In [10]:
visualize_ann(model,X.shape[1],output_dir='../ann_visualizer/output/')

'../ann_visualizer/output/20250327_211317_ann.html'

## Analysis

### Effect of depth (Number of layers) and Width (Number of neurons per layer)

#### Fixed Depth
- Number of hidden layers : 2
- Test 1 : 64 neurons per layer
- Test 2 : 128 neurons per layer
- Test 3 : 256 neurons per layer

##### Test 1

In [14]:
model_test_1_fixed_depth = NeuralNetwork('mse')
model_test_1_fixed_depth.add_layer(DenseLayer(output_size=128, activation=relu, init="Xavier"))
model_test_1_fixed_depth.add_layer(DenseLayer(output_size=64, activation=relu, init="Xavier"))
model_test_1_fixed_depth.add_layer(DenseLayer(output_size=10, activation=softmax, init="Xavier"))

model_test_1_fixed_depth.train(
    X_train,
    y_train,
    epochs=100,
    batch_size=64,
    learning_rate=0.05,
    isOne_hot=True,
    verbose=1,
    validation_data=(X_test, y_test)
)

[=================================================>] 100%
Epoch 1/100 - 3.03s - loss: 0.0472 - accuracy: 0.8773 - val_loss: 0.0209 - val_accuracy: 0.8736
[=================================================>] 100%
Epoch 2/100 - 2.85s - loss: 0.0175 - accuracy: 0.9012 - val_loss: 0.0157 - val_accuracy: 0.8975
[=================================================>] 100%
Epoch 3/100 - 2.67s - loss: 0.0144 - accuracy: 0.9130 - val_loss: 0.0136 - val_accuracy: 0.9096
[=================================================>] 100%
Epoch 4/100 - 3.21s - loss: 0.0128 - accuracy: 0.9193 - val_loss: 0.0127 - val_accuracy: 0.9163
[=================================================>] 100%
Epoch 5/100 - 3.02s - loss: 0.0118 - accuracy: 0.9257 - val_loss: 0.0117 - val_accuracy: 0.9219
[=================================================>] 100%
Epoch 6/100 - 2.94s - loss: 0.0109 - accuracy: 0.9310 - val_loss: 0.0110 - val_accuracy: 0.9276
[=================================================>] 100%
Epoch 7/100 - 2.84

{'loss': [0.047184172174571756,
  0.017456997277249607,
  0.014354399284281071,
  0.012842256548322497,
  0.011802394439441116,
  0.010922624635694801,
  0.01021323554874847,
  0.009500601850681878,
  0.008932191202756103,
  0.008422398925574831,
  0.007966618253229701,
  0.007531403731605003,
  0.007131747127303836,
  0.006779255397957534,
  0.006453089799938571,
  0.006158278321156281,
  0.005874351344794269,
  0.005640456968780611,
  0.005376614479449596,
  0.005155199235909341,
  0.0049508553737461075,
  0.004758971208682892,
  0.004573907879507656,
  0.004399023950075666,
  0.004247971977211571,
  0.004101145480680023,
  0.0039535427460946695,
  0.0038246552226254248,
  0.0036837433150429763,
  0.0035661973609378756,
  0.0034541174423023072,
  0.0033403453897131075,
  0.0032415226922945196,
  0.0031384275627202095,
  0.003035432398565227,
  0.002965290002988181,
  0.0028777677930909417,
  0.002775267099861831,
  0.002717364499977342,
  0.0026303523966053457,
  0.002566486145439477

In [11]:
predictions_test_1_fixed_depth = model_test_1_fixed_depth.predict(X_test)
pred_classes_test_1_fixed_depth = np.argmax(predictions_test_1_fixed_depth, axis=1)
accuracy_test_1_fixed_depth = accuracy_score(pred_classes_test_1_fixed_depth, y_test)
print("Test Accuracy:", accuracy_test_1_fixed_depth)

NameError: name 'model_test_1_fixed_depth' is not defined

In [20]:
X.shape[0]

70000

In [26]:
### Visualizer
visualize_ann(model_test_1_fixed_depth,X.shape[1])


ann_visualizer/output\interactive_neural_network.html


FileNotFoundError: [Errno 2] No such file or directory: 'ann_visualizer/output\\interactive_neural_network.html'

## Compare model with sklearn MLP

In [16]:
mlp = MLPClassifier(hidden_layer_sizes=(128, 64),activation='relu', 
                    solver='adam', max_iter=20, random_state=1, verbose=True)

mlp.fit(X_train, y_train)

Iteration 1, loss = 0.37724840
Iteration 2, loss = 0.15004672
Iteration 3, loss = 0.10562821
Iteration 4, loss = 0.08064147
Iteration 5, loss = 0.06428497
Iteration 6, loss = 0.05121985
Iteration 7, loss = 0.04219482
Iteration 8, loss = 0.03723277
Iteration 9, loss = 0.02986735
Iteration 10, loss = 0.02325205
Iteration 11, loss = 0.02124234
Iteration 12, loss = 0.01549812
Iteration 13, loss = 0.01433403
Iteration 14, loss = 0.01377085
Iteration 15, loss = 0.00927998
Iteration 16, loss = 0.00973711
Iteration 17, loss = 0.01097088
Iteration 18, loss = 0.00895768
Iteration 19, loss = 0.00620646
Iteration 20, loss = 0.00376964


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=20, random_state=1,
              verbose=True)

In [17]:
y_pred_mlp = mlp.predict(X_test)
acc_mlp = accuracy_score(y_test, y_pred_mlp)
print(f"MLPClassifier Test Accuracy: {acc_mlp:.4f}")

MLPClassifier Test Accuracy: 0.9790


In [18]:
print(classification_report(y_test, y_pred_mlp))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       986
           1       0.99      0.99      0.99      1125
           2       0.97      0.98      0.98       999
           3       0.99      0.95      0.97      1020
           4       0.98      0.98      0.98       975
           5       0.96      0.98      0.97       902
           6       0.99      0.99      0.99       982
           7       0.98      0.98      0.98      1042
           8       0.98      0.97      0.97       975
           9       0.95      0.99      0.97       994

    accuracy                           0.98     10000
   macro avg       0.98      0.98      0.98     10000
weighted avg       0.98      0.98      0.98     10000

